In [1]:
import random, gc, os, pickle

import datasets.utils
import models.utils
from models.cls_oml_ori_v2 import OML

import numpy as np

import higher
import torch
from torch.utils import data

# Constants

In [2]:
dataset_order_mapping = {
    1: [2, 0, 3, 1, 4],
    2: [3, 4, 0, 1, 2],
    3: [2, 4, 1, 3, 0],
    4: [0, 2, 1, 4, 3]
}
n_classes = 33
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
model_path = "saved_models/OML-2022-04-16_18-19-52.158289.pt"
# model_path = "saved_models/OML-2022-04-17_14-40-54.970677.pt"
# model_path = "saved_models/OML-2022-04-17_18-46-49.700106.pt"
# model_path = "saved_models/OML-2022-04-18_06-16-55.619110.pt"
model_path = "/data/model_runs/original_oml/OML-order1-id4-2022-07-18_17-53-13.518612.pt"
model_path = "/data/model_runs/original_oml/OML-order1-id3-2022-07-18_17-22-11.802295.pt"
model_path = "/data/model_runs/original_oml/OML-order2-id2-2022-07-18_19-54-42.201054.pt"
model_path = "/data/model_runs/original_oml/OML-order2-id4-2022-07-18_20-56-32.905358.pt"
model_path = "/data/model_runs/original_oml/OML-order1-id1-2022-07-18_16-26-09.710679.pt"


model_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456804.pt"



# adaptation_updates = 5
# adaptation_offset = 0
adaptation_updates_list = [2,5,10]
adaptation_offset_list = [0, 5, 100, 110, 120, 130, 140, 150, 200]

data_id_support =3
data_id_query = 1

use_db_cache = True
cache_dir = 'tmp'

In [3]:
args = {
    "order": 1,
    "n_epochs": 1,
    "lr": 3e-5,
    "inner_lr": 0.001*2,
    "meta_lr": 3e-5,
    "model": "bert",
    "learner": "oml",
    "mini_batch_size": 16,
    "updates": 5,
    "write_prob": 1.0,
    "max_length": 448,
    "seed": 42,
    "replay_rate": 0.01,
    "replay_every": 9600
}
updates = args["updates"]
mini_batch_size = args["mini_batch_size"]

In [4]:
torch.manual_seed(args["seed"])
random.seed(args["seed"])
np.random.seed(args["seed"])

# Load Dataset

In [5]:
if use_db_cache:
    test_dataset_support_file = os.path.join(cache_dir, f"{data_id_support}.cache")
    if os.path.exists(test_dataset_support_file):
        with open(test_dataset_support_file, 'rb') as f:
            test_dataset_support = pickle.load(f)
    else:
        test_dataset_support = datasets.utils.get_dataset_test("", data_id_support)
        test_dataset_support = datasets.utils.offset_labels(test_dataset_support)
        pickle.dump(test_dataset_support, open( test_dataset_support_file, "wb" ), protocol=pickle.HIGHEST_PROTOCOL)
        print(f"Pickle saved at {test_dataset_support_file}")
    print('Loaded Support Set {}'.format(test_dataset_support.__class__.__name__))
    
    if data_id_support == data_id_query:
        test_dataset_query = test_dataset_support
    else:
        test_dataset_query_file = os.path.join(cache_dir, f"{data_id_query}.cache")
        if os.path.exists(test_dataset_query_file):
            with open(test_dataset_query_file, 'rb') as f:
                test_dataset_query = pickle.load(f)
        else:
            test_dataset_query = datasets.utils.get_dataset_test("", data_id_query)
            test_dataset_query = datasets.utils.offset_labels(test_dataset_query)
            pickle.dump(test_dataset_query, open( test_dataset_query_file, "wb" ), protocol=pickle.HIGHEST_PROTOCOL)
            print(f"Pickle saved at {test_dataset_query_file}")
    print('Loaded Query Set {}'.format(test_dataset_query.__class__.__name__))
    
else:
    test_dataset_support = datasets.utils.get_dataset_test("", data_id_support)
    test_dataset_support = datasets.utils.offset_labels(test_dataset_support)
    print('Loaded Support Set {}'.format(test_dataset_support.__class__.__name__))

    if data_id_support == data_id_query:
        test_dataset_query = test_dataset_support
    else:
        test_dataset_query = datasets.utils.get_dataset_test("", data_id_query)
        test_dataset_query = datasets.utils.offset_labels(test_dataset_query)
    print('Loaded Query Set {}'.format(test_dataset_query.__class__.__name__))

Loaded Support Set DBPediaDataset
Loaded Query Set AmazonDataset


In [6]:
test_dataloader_support = data.DataLoader(test_dataset_support, batch_size=mini_batch_size, shuffle=False,
                                  collate_fn=datasets.utils.batch_encode)
test_dataloader_support_iter = iter(test_dataloader_support)

test_dataloader_query = data.DataLoader(test_dataset_query, batch_size=mini_batch_size, shuffle=False,
                                  collate_fn=datasets.utils.batch_encode)
test_dataloader_query_iter = iter(test_dataloader_query)

In [7]:
text, labels = next(iter(test_dataloader_support))
labels

[18, 18, 17, 15, 17, 11, 13, 13, 9, 20, 14, 15, 9, 12, 9, 22]

# Load Model

In [8]:
learner = OML(device=device, n_classes=n_classes, **args)
learner.load_model(model_path)

2022-08-01 10:17:04,506 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2022-08-01 10:17:05,739 - transformers.configuration_utils - INFO - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
2022-08-01 10:17:05,743 - transformers.configuration_utils - INFO - Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer

# Without Adaptation Evaluation

In [9]:
offset_arr = []
for adaptation_offset in adaptation_offset_list:
    print(f"Currently in adaptation offset of {adaptation_offset}")
    
    test_dataloader_support_iter = iter(test_dataloader_support)
    test_dataloader_query_iter = iter(test_dataloader_query)

    learner.rln.eval()
    learner.pln.eval()

    # For Offsetting dataset
    if adaptation_offset > 0:
        for _ in range(adaptation_offset):
            text, labels = next(test_dataloader_support_iter)
            text, labels = next(test_dataloader_query_iter)

    with higher.innerloop_ctx(learner.pln, learner.inner_optimizer,
                              copy_initial_weights=False,
                              track_higher_grads=False) as (fpln, diffopt):
        # Inner Loop
        support_set = []
        for _ in range(updates):
            text, labels = next(test_dataloader_support_iter)
            text, labels = next(test_dataloader_query_iter)
            support_set.append((text, labels))
        # Outer loop
        query_set = []
        text, labels = next(test_dataloader_query_iter)
        query_set.append((text, labels))
        for text, labels in query_set:
            labels = torch.tensor(labels).to(device)
            input_dict = learner.rln.encode_text(text)
            _repr = learner.rln(input_dict)
            output = fpln(_repr)
            loss = learner.loss_fn(output, labels)
            print(loss.item())
            pred = models.utils.make_prediction(output.detach())
            acc, prec, rec, f1 = models.utils.calculate_metrics(pred.tolist(), labels.tolist())

            print(acc, prec, rec, f1)
            offset_arr.append(acc)
print()
print("COPY PASTA")
for row in offset_arr:
    print(row)

Currently in adaptation offset of 0
1.1376738548278809
0.5625 0.6833333333333333 0.6 0.6186868686868687
Currently in adaptation offset of 5
1.1727596521377563
0.5 0.6033333333333333 0.5166666666666666 0.490952380952381
Currently in adaptation offset of 100
1.5657199621200562
0.625 0.7333333333333332 0.6666666666666666 0.6285714285714287
Currently in adaptation offset of 110
0.7064863443374634
0.75 0.7333333333333333 0.65 0.64
Currently in adaptation offset of 120
1.4243059158325195
0.5625 0.9166666666666666 0.6488095238095237 0.7333333333333333
Currently in adaptation offset of 130
1.1412097215652466
0.625 0.725 0.65 0.6005128205128205
Currently in adaptation offset of 140
0.9943897128105164
0.5 0.5733333333333335 0.5476190476190477 0.5421212121212121
Currently in adaptation offset of 150
1.5085012912750244
0.625 0.8452380952380952 0.5833333333333334 0.6506410256410255
Currently in adaptation offset of 200
0.9491214156150818
0.6875 0.77 0.7166666666666667 0.718095238095238

COPY PASTA


In [10]:
# del learner
# gc.collect()
# torch.cuda.empty_cache() 
# learner = OML(device=device, n_classes=n_classes, **args)
# learner.load_model(model_path)

# With Adaptation Evaluation

In [11]:
all_arr = []

for adaptation_offset in adaptation_offset_list:
    print()
    print(f"\tCurrently in adaptation offset of {adaptation_offset}")
    
    offset_arr = []
    for adaptation_updates in adaptation_updates_list:
        print(f"Currently in adaptation updates of {adaptation_updates}")
        learner.load_model(model_path)

        test_dataloader_support_iter = iter(test_dataloader_support)
        test_dataloader_query_iter = iter(test_dataloader_query)

        learner.rln.eval()
        learner.pln.train()

        # For Offsetting dataset
        if adaptation_offset > 0:
            for _ in range(adaptation_offset):
                text, labels = next(test_dataloader_support_iter)
                text, labels = next(test_dataloader_query_iter)

        with higher.innerloop_ctx(learner.pln, learner.inner_optimizer,
                                  copy_initial_weights=False,
                                  track_higher_grads=False) as (fpln, diffopt):
            # Inner Loop
            task_predictions, task_labels = [], []
            support_set = []
            query_set = []

            if adaptation_updates == 2:
                for ii in range(updates):
                    text, labels = next(test_dataloader_support_iter)
                    _, _ = next(test_dataloader_query_iter)
                    if ii < 2: 
                        support_set.append((text, labels))
                text, labels = next(test_dataloader_query_iter)
                query_set.append((text, labels))
            elif adaptation_updates == 5:
                for _ in range(updates):
                    text, labels = next(test_dataloader_support_iter)
                    _, _ = next(test_dataloader_query_iter)
                    support_set.append((text, labels))
                text, labels = next(test_dataloader_query_iter)
                query_set.append((text, labels))
            elif adaptation_updates == 10:
                for _ in range(updates):
                    text, labels = next(test_dataloader_support_iter)
                    _, _ = next(test_dataloader_query_iter)
                    support_set.append((text, labels))
                text, labels = next(test_dataloader_query_iter)
                query_set.append((text, labels))
                for _ in range(updates):
                    text, labels = next(test_dataloader_support_iter)
                    support_set.append((text, labels))
            else:
                raise Exception("ERROR adaptation_updates")

            for text, labels in support_set:
                labels = torch.tensor(labels).to(device)
                input_dict = learner.rln.encode_text(text)
                _repr = learner.rln(input_dict)
                output = fpln(_repr)
                loss = learner.loss_fn(output, labels)
#                 print(loss.item())
                diffopt.step(loss)
                pred = models.utils.make_prediction(output.detach())
                task_predictions.extend(pred.tolist())
                task_labels.extend(labels.tolist())

            acc, prec, rec, f1 = models.utils.calculate_metrics(task_predictions, task_labels)
#             print(acc, prec, rec, f1)

            # Outer loop
            for text, labels in query_set:
                labels = torch.tensor(labels).to(device)
                input_dict = learner.rln.encode_text(text)
                _repr = learner.rln(input_dict)
                output = fpln(_repr)
                loss = learner.loss_fn(output, labels)
#                 print(loss.item())
                pred = models.utils.make_prediction(output.detach())
                acc, prec, rec, f1 = models.utils.calculate_metrics(pred.tolist(), labels.tolist())

                print(acc, prec, rec, f1)
                offset_arr.append(acc)
    all_arr.append(offset_arr)

print("COPY PASTA")
for row in all_arr:
    print("\t".join(str(i) for i in row))


	Currently in adaptation offset of 0
Currently in adaptation updates of 2
0.5625 0.6833333333333333 0.6 0.6186868686868687
Currently in adaptation updates of 5
0.5625 0.6833333333333333 0.6 0.6186868686868687
Currently in adaptation updates of 10
0.5625 0.6833333333333333 0.6 0.6186868686868687

	Currently in adaptation offset of 5
Currently in adaptation updates of 2
0.5 0.6033333333333333 0.5166666666666666 0.490952380952381
Currently in adaptation updates of 5
0.5 0.6033333333333333 0.5166666666666666 0.490952380952381
Currently in adaptation updates of 10
0.5 0.6033333333333333 0.5166666666666666 0.490952380952381

	Currently in adaptation offset of 100
Currently in adaptation updates of 2
0.625 0.7333333333333332 0.6666666666666666 0.6285714285714287
Currently in adaptation updates of 5
0.625 0.7333333333333332 0.6666666666666666 0.6285714285714287
Currently in adaptation updates of 10
0.625 0.7333333333333332 0.6666666666666666 0.6285714285714287

	Currently in adaptation offset

In [12]:
for t,p,l in zip(text, pred.tolist(), labels.tolist()):
    print(t)
    print(f"Predicted\t{p}")
    print(f"Actual\t{l}")
    print()

filthy divine kicks ass. ok, i like it! they have a great diversity of sound and they really kick ass! the guitar work borders on brilliant and their lead singer is total eye candy! i'm anxiously awaiting their next release - c'mon guys, when's it coming out?
Predicted	4
Actual	4

very good cd, almost as good as soundbombing two. i've been listening to the cd for 2 days now and i really like most of the songs off of it. some aren't my style but they're still good. this cd's almost as good as the soundbombing 2 cd. definately worth the money.
Predicted	3
Actual	3

poorly mixed. the main problem i have with this cd is that it seems poorly mixed. there is no sonic depth to it - all the sounds are flat and hard to distinguish from each other.
Predicted	1
Actual	1

adopting a chinese daughter prior to the ccca. baby in a box is the story of one missionary family's struggle to bring together an abandoned chinese daughter and an american family. the union takes place in china prior to the est